# Bài 5: Train/Test với K-Fold và Đánh giá Mô hình (SVM, Logistic Regression)

## Yêu cầu:
- Thực hiện train/test theo K-Fold
- Tính các metrics: Accuracy, Precision, Recall, F1-Score
- So sánh SVM và Logistic Regression
- Phân tích kết quả chi tiết

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

## 1. Load và Chuẩn bị Dữ liệu

In [ ]:
# Load dataset (có thể thay đổi dataset)
# Chọn 1 trong 3: load_iris(), load_wine(), load_breast_cancer()
data = load_iris()
X = data.data
y = data.target

print(f"Dataset: {data.get('DESCR', '').split('\n')[0]}")
print(f"Số mẫu: {len(X)}")
print(f"Số đặc trưng: {X.shape[1]}")
print(f"Số lớp: {len(np.unique(y))}")
print(f"Phân bố các lớp: {Counter(y)}")
print(f"\nTên các đặc trưng: {data.feature_names}")
print(f"Tên các lớp: {data.target_names}")

### 1.1. Chuẩn hóa dữ liệu

In [ ]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nThống kê trước chuẩn hóa:")
print(f"Mean: {X.mean(axis=0)}")
print(f"Std:  {X.std(axis=0)}")

print("\nThống kê sau chuẩn hóa:")
print(f"Mean: {X_scaled.mean(axis=0)}")
print(f"Std:  {X_scaled.std(axis=0)}")

## 2. Thiết lập K-Fold Cross Validation

In [ ]:
# Thiết lập K-Fold
K = 5
cv = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

print(f"\nSử dụng {K}-Fold Stratified Cross Validation")
print(f"Mỗi fold sẽ có khoảng {len(X)//K} mẫu test")

## 3. Định nghĩa Mô hình

In [ ]:
# Định nghĩa các mô hình
models = {
    'SVM (Linear)': SVC(kernel='linear', random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', random_state=42),
    'SVM (Poly)': SVC(kernel='poly', degree=3, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

print("\nCác mô hình sẽ được đánh giá:")
for name in models.keys():
    print(f"  - {name}")

## 4. Thực hiện K-Fold Cross Validation

In [ ]:
# Định nghĩa các metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision_macro',
    'recall': 'recall_macro',
    'f1': 'f1_macro'
}

# Dictionary để lưu kết quả
results = {}

# Đánh giá từng mô hình
print("\n" + "="*80)
print("ĐÁNH GIÁ MÔ HÌNH VỚI K-FOLD CROSS VALIDATION")
print("="*80)

for model_name, model in models.items():
    print(f"\n{model_name}:")
    print("-" * 60)
    
    # Cross validate
    cv_results = cross_validate(
        model, X_scaled, y, 
        cv=cv, 
        scoring=scoring,
        return_train_score=True,
        n_jobs=-1
    )
    
    # Lưu kết quả
    results[model_name] = cv_results
    
    # In kết quả
    for metric in ['accuracy', 'precision', 'recall', 'f1']:
        train_scores = cv_results[f'train_{metric}']
        test_scores = cv_results[f'test_{metric}']
        
        print(f"\n{metric.upper()}:")
        print(f"  Train: {train_scores.mean():.4f} (+/- {train_scores.std() * 2:.4f})")
        print(f"  Test:  {test_scores.mean():.4f} (+/- {test_scores.std() * 2:.4f})")
        print(f"  Scores per fold: {[f'{s:.4f}' for s in test_scores]}")

## 5. So sánh Kết quả các Mô hình

In [ ]:
# Tạo DataFrame để so sánh
comparison_data = []

for model_name, cv_results in results.items():
    row = {'Model': model_name}
    for metric in ['accuracy', 'precision', 'recall', 'f1']:
        test_scores = cv_results[f'test_{metric}']
        row[f'{metric.capitalize()} (mean)'] = test_scores.mean()
        row[f'{metric.capitalize()} (std)'] = test_scores.std()
    comparison_data.append(row)

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.set_index('Model')

print("\n" + "="*80)
print("BẢNG SO SÁNH KẾT QUẢ")
print("="*80)
print(df_comparison.to_string())

# Tìm mô hình tốt nhất cho mỗi metric
print("\n" + "="*80)
print("MÔ HÌNH TỐT NHẤT CHO MỖI METRIC")
print("="*80)
for metric in ['Accuracy', 'Precision', 'Recall', 'F1']:
    best_model = df_comparison[f'{metric} (mean)'].idxmax()
    best_score = df_comparison[f'{metric} (mean)'].max()
    print(f"{metric:10s}: {best_model:25s} ({best_score:.4f})")

## 6. Visualize Kết quả

In [ ]:
# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('K-Fold Cross Validation Results Comparison', fontsize=16, fontweight='bold')

metrics = ['accuracy', 'precision', 'recall', 'f1']
positions = [(0, 0), (0, 1), (1, 0), (1, 1)]

for metric, pos in zip(metrics, positions):
    ax = axes[pos]
    
    # Chuẩn bị dữ liệu
    data_to_plot = []
    labels = []
    
    for model_name in models.keys():
        scores = results[model_name][f'test_{metric}']
        data_to_plot.append(scores)
        labels.append(model_name)
    
    # Box plot
    bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True)
    
    # Tô màu
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    # Scatter plot các điểm
    for i, scores in enumerate(data_to_plot, 1):
        y = scores
        x = np.random.normal(i, 0.04, size=len(y))
        ax.scatter(x, y, alpha=0.5, s=30)
    
    ax.set_ylabel('Score', fontsize=11)
    ax.set_title(f'{metric.upper()}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xticklabels(labels, rotation=15, ha='right')

plt.tight_layout()
plt.show()

## 7. Đánh giá Chi tiết trên Toàn bộ Dataset

In [ ]:
# Train trên toàn bộ dataset và đánh giá
print("\n" + "="*80)
print("ĐÁNH GIÁ CHI TIẾT (Train trên toàn bộ dataset)")
print("="*80)

for model_name, model in models.items():
    print(f"\n{model_name}:")
    print("-" * 60)
    
    # Train
    model.fit(X_scaled, y)
    
    # Predict
    y_pred = model.predict(X_scaled)
    
    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y, y_pred, target_names=data.target_names))
    
    # Confusion Matrix
    cm = confusion_matrix(y, y_pred)
    
    # Vẽ Confusion Matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=data.target_names,
                yticklabels=data.target_names)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

## 8. Phân tích Kết quả theo từng Fold

In [ ]:
# Chọn mô hình tốt nhất để phân tích chi tiết
best_model_name = df_comparison['Accuracy (mean)'].idxmax()
best_model = models[best_model_name]

print(f"\nPhân tích chi tiết cho mô hình: {best_model_name}")
print("="*80)

# Thực hiện K-Fold thủ công để phân tích từng fold
fold_results = []

for fold, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
    # Chia dữ liệu
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Train
    best_model.fit(X_train, y_train)
    
    # Predict
    y_pred = best_model.predict(X_test)
    
    # Tính metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    fold_results.append({
        'Fold': fold,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Train Size': len(train_idx),
        'Test Size': len(test_idx)
    })

# Tạo DataFrame
df_folds = pd.DataFrame(fold_results)
print("\nKết quả từng Fold:")
print(df_folds.to_string(index=False))

# Thống kê
print("\nThống kê:")
print(df_folds[['Accuracy', 'Precision', 'Recall', 'F1-Score']].describe())

## 9. Kết luận và Khuyến nghị

In [ ]:
print("\n" + "="*80)
print("KẾT LUẬN")
print("="*80)

# Tìm mô hình tốt nhất
best_overall = df_comparison['F1 (mean)'].idxmax()
best_f1 = df_comparison.loc[best_overall, 'F1 (mean)']

print(f"\n1. Mô hình tốt nhất (theo F1-Score): {best_overall}")
print(f"   - F1-Score: {best_f1:.4f}")
print(f"   - Accuracy: {df_comparison.loc[best_overall, 'Accuracy (mean)']:.4f}")
print(f"   - Precision: {df_comparison.loc[best_overall, 'Precision (mean)']:.4f}")
print(f"   - Recall: {df_comparison.loc[best_overall, 'Recall (mean)']:.4f}")

print("\n2. Nhận xét:")
print("   - K-Fold Cross Validation giúp đánh giá mô hình một cách toàn diện")
print("   - Stratified K-Fold đảm bảo phân bố cân bằng giữa các fold")
print("   - Cần xem xét cả 4 metrics: Accuracy, Precision, Recall, F1-Score")

print("\n3. Khuyến nghị:")
print("   - Sử dụng mô hình", best_overall, "cho bài toán này")
print("   - Có thể thử tuning hyperparameters để cải thiện hiệu suất")
print("   - Nên thu thập thêm dữ liệu nếu có thể")

## Bài tập thực hành

1. Thử nghiệm với các dataset khác (load_wine, load_breast_cancer)
2. Thay đổi số fold (K=3, K=10) và so sánh kết quả
3. Thử các kernel khác của SVM (sigmoid, precomputed)
4. Tune hyperparameters (C, gamma cho SVM; penalty, C cho Logistic Regression)
5. Thêm các mô hình khác (Decision Tree, Random Forest, KNN)
6. Phân tích các trường hợp bị phân loại sai
7. Vẽ ROC curve cho các mô hình (với binary classification)
8. So sánh thời gian training của các mô hình